In [1]:
import pandas as pd
from dataset import COVIDDataset
from utils.etc import split_df, collate_fn
from utils.augmentation import get_study_transform
import torchvision.transforms as T
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from pl_script.unet_smp import LitUnetSmp
from pytorch_lightning.core.datamodule import LightningDataModule
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch

In [2]:
class MyDataModule(LightningDataModule):
    def __init__(self):
        super().__init__()
    # def setup(self, stage):
    def test_dataloader(self):
        root = "./data-512"
        train_df = pd.read_csv(os.path.join(root, "merged_drop_multi_image_study.csv"))
        train_df, val_df = split_df(train_df, 8888, 0)
        _, transform = get_study_transform(512)
        dataset = COVIDDataset(root=root, df=train_df[:2], img_size=512, mask="both", transform=transform, mask_img_size=128, use_cache=False)
        return DataLoader(dataset, batch_size=2, collate_fn=collate_fn, pin_memory=True, num_workers=8)

In [3]:
filename = "densenet169-512-aug-epoch=04-sgkf4-val_loss=0.7285.ckpt"

In [4]:
target_ckpt_path = f"/root/siim-covid/pl_output/{filename}"
backbone_name = target_ckpt_path.split("/")[-1].split("-")[0].replace("_","-")
config_dict = OmegaConf.merge(OmegaConf.load("config.yaml"), OmegaConf.load("model/unet_smp.yaml"))
config_dict.model = "unet_smp"
config_dict.unet_smp.backbone_name = backbone_name
config_dict.unet_smp.classes = 2

model = LitUnetSmp.load_from_checkpoint(target_ckpt_path, config=config_dict)

In [5]:
trainer = pl.Trainer(
    # precision=16,
    deterministic=True,
    check_val_every_n_epoch=1,
    gpus=[1],
    weights_summary="top",
    # accelerator='ddp_spawn',
    # plugins=DDPPlugin(find_unused_parameters=False),
)
model.freeze()
trainer.test(model=model, datamodule=MyDataModule())

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/root/siim-covid/utils/augmentation.py:92: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/root/anaconda3/envs/siim-covid/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:645: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/root/siim-covid/utils/augmentation.py:92: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/root/anaconda3/envs/siim-covid/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:645: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


Testing: 0it [00:00, ?it/s]

/root/siim-covid/pl_script/unet_smp.py:202: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(outputs))
/root/anaconda3/envs/siim-covid/lib/python3.7/site-packages/pytorch_lightning/core/step_result.py:115: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)
tensor([[0.1664, 0.3153, 0.3093, 0.2090],
        [0.0344, 0.0132, 0.0465, 0.9058]], device='cuda:1')
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.518170177936554,
 'test_loss_cls': 0.6265528798103333,
 'test_loss_seg': 0.40978750586509705,
 'test_mAP': 0.4999995231628418,
 'test_mAP_epoch': 0.4999995231628418}
-----------------------------------------------------

[{'test_loss_cls': 0.6265528798103333,
  'test_mAP': 0.4999995231628418,
  'test_loss_seg': 0.40978750586509705,
  'test_loss': 0.518170177936554,
  'test_mAP_epoch': 0.4999995231628418}]

In [6]:
model_path = f"/root/siim-covid/output/study-unet-smp/{filename}"

In [7]:
from model.unet_smp import SMPModel

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
config = dotdict({
    "model_name": "unet_smp",
    "unet_smp": dotdict({
      "backbone_name": model_path.split("/")[-1].split("-")[0].replace("_","-"),
      "model_type": "unetplusplus",
      "neck_type": "D",
      "gem_pooling": False,
      "hidden_dim": 512,
      "encoder_weights": "imagenet",
      "mask_type": "lung",
      "decoder_blocks": 3,
      "decoder_channels": None,
      "classes": 2,
    }),
})

model = SMPModel(config.unet_smp)
model.load_state_dict(torch.load(model_path)) 
model.to("cuda:1")
model.eval()
None

In [10]:
root = "./data-512"
train_df = pd.read_csv(os.path.join(root, "merged_drop_multi_image_study.csv"))
train_df, val_df = split_df(train_df, 8888, 0)
_, transform = get_study_transform(512)
dataset = COVIDDataset(root=root, df=train_df[:2], img_size=512, mask="both", transform=transform, mask_img_size=128, use_cache=False)

imgs, _, _, boxes, targets_image, targets_study, masks = dataset[0]
imgs = imgs.float().unsqueeze(0)
imgs = imgs.to("cuda:1")
F.softmax(model(imgs))

/root/siim-covid/utils/augmentation.py:92: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/root/anaconda3/envs/siim-covid/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:645: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,
ipykernel_launcher:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.1664, 0.3153, 0.3093, 0.2090]], device='cuda:1',
       grad_fn=<SoftmaxBackward>)

In [11]:
train_df[:2]

,Unnamed: 0,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,PatientID
0,0,65761e66de9f_image,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study,0,1,0,0,7d3bf0aef747
1,1,51759b5579bc_image,NaN,none 1 0 0 1 1,000c9c05fd14_study,0,0,0,1,2c00dc1ead80


In [13]:
train_df[:2]

,Unnamed: 0,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,PatientID
0,0,65761e66de9f_image,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852_study,0,1,0,0,7d3bf0aef747
1,1,51759b5579bc_image,NaN,none 1 0 0 1 1,000c9c05fd14_study,0,0,0,1,2c00dc1ead80
